# Tracking problem with Linear Quadratic Control


The goal of this Live Script is to control a particle modeled by a simple 
linear model with disturbances in order to track an ellipsoidal reference trajectory 
such that the variance of the error between the actual and the reference trajectory 
is minimized. The model of the particle is

$$\begin{array}{l} \dot{z} = u_z \\ \dot{u_z} = -c_f\, u_z + u_z \\ \dot{y}  
= u_y \\ \dot{u_y} = -c_f\, u_y + u_y\end{array}$$

where $z$ and $y$ are the coordinates in a fixed world frame, $v_z$ and $v_y$ 
are the velocities expressed also in the fixed world frame and $u_z$, $u_y$ 
are the applied forces (normalized by the mass); $c_f$ is a friction constant 
(normalized by the mass). The model can be discretized at a given sampling period 
$\tau$ ; assuming that the actuation $u_z$ and $u_y$ is hold constant between 
sampling times $t_k = k_{\tau}$ this leads to

$$x_{k+1} = Ax_k + Bu_k$$

where $x_k =[z(t_k)\quad  v_z(t_k) \quad y(t_k)\quad v_y(t_k)]^T$ and $u_k 
= [u_z(t_k)\quad u_y(t_k)]$, is the actuation vector in the interval $[t_k,\, 
t_{k+1})$. The reference for the components of $x_k$ corresponding to positions 
$z$, $y$ is the following

$$(z_k^r, \, y_k^r) = (a\, \cos (\frac{2\pi k}{T}), b\,  \sin (\frac{2\pi 
k}{T}))$$

where $a$ and $b$ are given constants that shape the ellipsoid. 

Let us first discuss how to compute the unique initial velocities $(v_x(0),\, 
v_y(0))$, and the unique numerical values of the control input in order to ensure 
that $z_k = z_k^r$ and $y_k = y_k^r$ for every $k\in \{0,1,....,T-1\}$, and 
$(v_{z,T}, v_{y,T}) = (v_{z,0}, v_{y,0})$, where $z_k=z(t_k)$ and $y_k=y(t_k)$, 
$v_{z,k}=v_z(t_k)$, $v_{y,k}=v_y(t_k)$. If the control input is denoted by $u_0^r, 
\, u_1^r,\, ....... \, u_{T-1}^r$, then

$$x^r_{k+1} = Ax^r_k + Bu^r_k$$

for $k \in \{0,...,T-1\}$and $x_k^r =[z_k^r\quad  v_{z,k}^r \quad y_k^r\quad 
v_{y,k}^r]^T$. For $k\geq T$ define $u_k^r$ as $u_{lT+k}^r$ for every $k\in 
\{0,1,....,T-1\}$and $l\in N_0$. Thus, these periodic control values, lead to 
periodic state variables if we iterate $x^r_{k+1} = Ax^r_k + Bu^r_k$, for $x_k^r=x_{lT+k}^r$, 
for every $k\in \{0,1,....,T-1\}$and $l\in N_0$.

Note that to compute these values it suffices to solve a linear system of 
equations of $2T+2$ equations with $2T+2$ unknowns, namely $u_0^r, \, u_1^r,\, 
....... \, u_{T-1}^r$ and $(v_x(0),\, v_y(0))$. In fact from the system dynamics 
we can write the following $2T$ equations

$$ \underbrace{\left[\matrix{ z^r_1 \cr y^r_1 \cr \vdots \cr z^r_{T} \cr 
y^r_T }\right]}_F=\underbrace{\left[\matrix{ C & 0 & \dots & 0 &0\cr 0 & C & 
0 &\dots & 0 \cr \dots & \dots & \dots &\dots& \dots \cr 0 & 0 &\dots & 0 & 
C}\right]}_{\bar{C}} (\underbrace{\left[\matrix{ A \cr A^2 \cr \vdots  \cr A^{T-1} 
}\right]}_{\bar{A}}(\underbrace{\left[\matrix{a \cr 0 \cr 0 \cr 0}\right]}_D+\underbrace{\left[\matrix{0& 
0 \cr 1 & 0 \cr 0 & 0 \cr 0 & 1 }\right]}_E \underbrace{\left[\matrix{ v_x(0)\cr 
v_y(0)}\right]}_{X_2})+\left[\matrix{ B & 0 & 0&\dots & 0  \cr AB & B & 0 & 
\dots & 0 \cr \dots & \dots & \dots & \dots & \dots \cr A^{T-2}B & A^{T-3}B 
& \dots & AB & B }\right]_{\bar{B}}\underbrace{\left[\matrix{ u_0^r \cr u_1^r 
\cr \dots \cr u_{T-1}^r}\right]}_{X_1})$$

and the following two equations

$$ {\left[\matrix{ v_x(0)\cr v_y(0)}\right]}=\underbrace{\left[\matrix{0 
& 1 & 0 & 0 \cr 0 & 0 & 0 & 1} \right]}_\Pi (A^{T-1}(\left[\matrix{a \cr 0 \cr 
0 \cr 0}\right]+\left[\matrix{0& 0 \cr 1 & 0 \cr 0 & 0 \cr 0 & 1 }\right] \left[\matrix{ 
v_x(0)\cr v_y(0)}\right])+\left[\matrix{  A^{T-2}B & A^{T-3}B & \dots & AB & 
B }\right]{\left[\matrix{ u_0^r \cr u_1^r \cr \dots \cr u_{T-1}^r}\right]})$$

and where the unknows are now denoted by $X_1$ and $X_2$. In a compact form

$$\left[ \matrix{ \bar{C}\bar{B} & \bar{C}\bar{A}E  \cr \bar{\Pi}\bar{B} & 
\bar{\Pi}\bar{A}E -I} \right   ]\left[ \matrix{ X_1 \cr X_2} \right   ]=\left[ 
\matrix{ F-\bar{C}\bar{A}D \cr -\bar{\Pi}\bar{A}D} \right   ]$$

where $\bar{\Pi}=\left[ \matrix{0 & \Pi} \right]$.

The matlab function `getfeedforward` below implements these equations to provide 
the control inputs and the initial velocity which lead to the tracking of a 
periodic elipsoidal trajectory. The outputs of this function are the matrices 
$U_r$ and $X_r$ with dimensions $2\times N$ and $4\times N$, respectively, 
such that Ur(1:2,k) = $u_{k-1}^r$ and Xr(1:4,k)= $x_{k-1}^r$ given the periodic 
T, the sampling period $\tau$ , the parameters of the ellipsoid $a, \, b$, the 
parameter of the model $c_f$ and the number of desired time steps $N$.

Let us now assume that the discrete-time model is subject to have zero-mean 
stochastic disturbances $\omega _k$ with covariance $\mathrm{E} [\omega _k \omega 
_k ^T] = W_k$, so that $x_{k+1} = Ax_k + Bu_k + \omega _k$. If we set $e_k = 
x_k - x_k^r$ and $v_k = u_k - u_k^r$, we can write an error system as $e_{k+1} 
= Ae_k + Bv_k + \omega _k$. The goal is to design a control policy for this 
error system that minimizes the following mean square error over a single period, 
that is, 

$$\mathrm{E}[\sum_{k=0}^{T-1}(C_ze_k)^2 + (C_ye_k)^2]$$

where $C_z = [1\quad 0\quad 0\quad 0]$ and $C_z = [0\quad 0\quad 1\quad 0]$ 
and a control policy that minimizes an average cost

$$\lim_{r\rightarrow \infty} \frac{1}{r} \mathrm{E}[\sum_{k=0}^{rT-1}(C_ze_k)^2 
+ (C_ye_k)^2]$$

Note that the control input is not penalized. Still after applying dynamic 
programming we can conclude that the optimal policy for the control input $v_k$ 
trivially try to set the output errors at time $k+1$ , $C_ze_k$ and $C_ye_k$ 
to zero, and is given by

$$v_k=\left[\matrix{-\frac{1}{B_{11}}A_{1,:} \cr      -\frac{1}{B_{32}}A_{3,:}}\right]$$

where $A_{i,:}$denotes row $i$ of matrix $A$ and $B_{ij}$ denotes component 
$i,j$ or matrix $B$.

These policies are implemented in the MATLAB function `lqcontrol` below. The 
outputs of the function are the state and control input for known input distrurbances 
$\omega _k$ and the initial state $x_0$. Additionally, the policy now is $u_k 
= u_k^r + v_k$, where $u_k^r$ is computed from the previous function, the feedforward. 
The opt parameter of the function is set to one if we want to use the finite 
horizon policy. Finally, as before, the output of the function are the $U_r$ 
and $X_r$ matrices with dimensions 2xN and 4xN, respectively, such that U(1:2,k) 
= $u_{k-1}$ and X(1:4,k)= $x_{k-1}$. 

An example illustrating how this function works is given next.

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import clear_output

In [ ]:
def getfeedforward(T,tau,a,b,cf,N):

    # define model matrices and discretization
    Ac= np.array([[0, 1,   0, 0],
                [0, -cf, 0, 0],
                [0, 0,   0, 1],
                [0, 0,   0, -cf]])
    Bc= np.array([[0, 0],
                [1, 0], 
                [0, 0],
                [0, 1]])
    Cc = np.array([[1, 0, 0, 0],
                    [0, 0, 1, 0]])
    Dc = np.array([[0, 0], [0, 0]])

    A, B, C = signal.cont2discrete((Ac, Bc, Cc, Dc), tau)[:3]

    # define matrices needed to solve linear system, according to the description above
    Faux = np.vstack((a*np.cos(2*np.pi*np.arange(1,T+1)/T),b*np.sin(2*np.pi*np.arange(1,T+1)/T)))
    F    = (np.kron(Faux[0,:],[1, 0]) + np.kron(Faux[1,:],[0, 1])).reshape(20,1)
    Cbar = np.kron(np.eye(T),C) 
    D = np.array([[a], [0], [0], [0]])
    E = np.array([[0, 1, 0, 0], [0, 0, 0, 1]]).T
    Abar = np.zeros((4*T,4))
    Bbar = np.zeros((4*T,2*T))
    for i in range(T):
        Abar[np.arange(i*4,4*(i+1)),:] = np.linalg.matrix_power(A,i+1)
        for j in range(i+1):
            Bbar[i*4:4*(i+1),j*2:2*(j+1)] = np.linalg.matrix_power(A,(i+1)-(j+1))@B
        
    Pi = np.array([[0, 1, 0, 0],[0, 0, 0, 1]])
    Pibar = np.zeros((2,4*T))
    Pibar[:,36:40] = Pi
    B_ = np.vstack((F-Cbar@Abar@D,-Pibar@Abar@D))
    A_ = np.vstack((np.hstack((Cbar@Bbar, Cbar@Abar@E)),np.hstack((Pibar@Bbar, Pibar@Abar@E-np.eye(2)))))

    # solve linear system and extract solution 
    X_ = np.linalg.lstsq(A_, B_, rcond=None)
    u = np.zeros((2,T))
    for k in range(T):
        u[:,[k]] = X_[0][2*k:2*(k+1)]
    
    V0 = X_[0][2*T:2*T+2]

    # iterate x_k+1=A x_k + B uk 
    Xr = np.zeros((4,N))
    Ur = np.zeros((2,N))
    Xr[:,[0]] = np.array([a, V0[0][0], 0, V0[1][0]]).reshape(4,1)
    for k in range(N):
        Ur[:,[k]] = u[:,[(k%T)]]
        if k+1 < N:
            Xr[:,[k+1]] = A@Xr[:,[k]]+B@Ur[:,[k]]
        
    return Ur, Xr

In [ ]:
def lqcontrol(T,tau,a,b,cf,N,x0,w):

    # define model matrices
    Ac= np.array([[0, 1, 0, 0],
                [0, -cf, 0, 0],
                [0, 0,   0, 1],
                [0, 0,   0, -cf]])
    Bc= np.array([[0, 0],
                    [1, 0], 
                    [0, 0],
                    [0, 1]])
    Cc = np.array([[1, 0, 0, 0],
                    [0, 0, 1, 0]])
    Dc = np.array([[0, 0],[0, 0]])
    A, B, C = signal.cont2discrete((Ac, Bc, Cc, Dc), tau)[:3]

    # obtain gains of the optimal policy (same for infinite/finite horizon)
    K = np.vstack((-1/B[0,0]*A[0,:],-1/B[2,1]*A[2,:]))
    
    # obtain feedforward 
    [Ur,Xr] = getfeedforward(T,tau,a,b,cf,N)

    # iterate x_k+1=A x_k + B uk +wk
    X = np.zeros((4,N))
    U = np.zeros((2,N))
    X[:,[0]] = x0
    for k in range(N):
        U[:,[k]] = Ur[:,[k]]+K@(X[:,[k]]-Xr[:,[k]])
        if k+1 < N:
            X[:,[k+1]] = A@X[:,[k]]+B@U[:,[k]]+w[:,[k]]
        
    return U, X

In [ ]:
# define the parameters of the problem
tau = 1
T   = 10
a   = 2
b   = 1
cf  = 0.1
N   = 16
x0  = np.array([[1], [0.0333], [0], [1.9994]])
np.random.seed(1)
# generate disturbances
w = 0.1 * np.random.randn(4, N) 
# compute control U and state values X given disturbances
U, X = lqcontrol(T,tau,a,b,cf,N,x0,w)
Ur, Xr = getfeedforward(T,tau,a,b,cf,N); # this function is called here for the animation

for k in range(N):
    f = plt.figure()
    ax = plt.gca()
    clear_output(wait=True)
    ax.scatter(Xr[0,:], Xr[2,:],label="Reference")
    ax.scatter(X[0,:], X[2,:], label="actual trajectory")
    XANIM = 0.1*np.array([1,-1,-1,1])
    YANIM = 0.1*np.array([1,1,-1,-1])
    x = X[0,k] + XANIM
    y = X[2,k] + YANIM
    dd = patches.Polygon(np.column_stack((x,y)))
    ax.add_patch(dd)
    ax.legend()
    plt.pause(tau)